In [1]:
import pandas as pd
import numpy as np
from itertools import repeat
from keras.models import Sequential
from keras.layers import Dense
import keras
from tensorflow.keras import backend
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


New Test

In [2]:
df = pd.DataFrame()
df  = pd.read_csv('Xtrain.csv',encoding='utf-8',header=None)
df.head()
X_train = np.asarray(df)
print("Matrix of dimension: (",X_train.shape[0],",",X_train.shape[1],")")

Matrix of dimension: ( 6182 , 3810 )


In [3]:
df  = pd.read_csv('Xtest.csv',encoding='utf-8',header=None)
df.head()
X_test = np.asarray(df)
print("Matrix of dimension: (",X_test.shape[0],",",X_test.shape[1],")")

Matrix of dimension: ( 2647 , 3810 )


In [4]:
df  = pd.read_csv('nYtest.csv',encoding='utf-8',header=None)
Y_test = df[0]
Y_test[Y_test == -1] = 0
df.head()

,0
0,0
1,0
2,1
3,0
4,0


In [5]:
df = pd.DataFrame()
df  = pd.read_csv('Stest.csv',encoding='utf-8',header=None)
S_test = np.asarray(df) #Super mega importante asarray, sino copia la matriz de otra forma
df.head()

,0,1,2,3,4,5,6,7,8,9,...,7454,7455,7456,7457,7458,7459,7460,7461,7462,7463
0,3227,794,2939,3365,734,2802,1021,2693,2547,2357,...,0,0,0,0,0,0,0,0,0,0
1,1463,3650,145,77,2882,2422,1241,623,1856,976,...,0,0,0,0,0,0,0,0,0,0
2,220,1306,2791,3611,1651,253,1821,3758,322,1109,...,0,0,0,0,0,0,0,0,0,0
3,2787,845,262,1911,3009,1998,320,1820,280,2068,...,0,0,0,0,0,0,0,0,0,0
4,2787,845,262,1911,3150,3359,1637,3227,3628,2532,...,0,0,0,0,0,0,0,0,0,0


In [6]:
print(S_test.shape)
print(S_test[0])

(2647, 7464)
[3227  794 2939 ...    0    0    0]


In [7]:
df = pd.DataFrame()
df  = pd.read_csv('nYtrain.csv',encoding='utf-8',header=None)
Y_train = df[0]
Y_train[Y_train == -1] = 0
df.head()

,0
0,0
1,0
2,0
3,0
4,1


PBR

In [8]:
def csa (X,Y,XT):
    num_docs = X.shape[0] # number of trainning docs
    num_terms = X.shape[1]
    num_class = 2
    TR = np.zeros((num_terms, num_class))
    DL = np.sum(X, axis=1) # sum each elemet row, wich is each df element of a document
    DL[DL == 0] = 1 #Those documents with not terms in dicc, are empty .
   
    # Non - Normalization 0'
    for j in range (0,num_class): # j = [0;1]
        docs = np.where(Y == j)[0] 
        for i in range(0,num_terms):
            tf_v = X[docs,i] # tf_v is tern frecuency of each term
            TR[i][j] = sum(np.log2(1 + tf_v / DL [docs]))            
    print("PRB: (",TR.shape[0],",",TR.shape[1],")")

    # Normalization 1' Sum all the weights vectors 
    n1 = np.sum(TR, axis = 0) #Sum by colum
    n1[n1 == 0] = 1
    Tik = TR / n1

    # Normalization 2' Each componet of tik [ti1,ti2]/ [ti1+ti2;ti1+ti2]
    for i in range(0,num_terms):
        suma = sum(Tik[i])
        if suma == 0:
            suma = 1
        Tik[i] = Tik[i] / suma # Tik[i] = Tik[i] / sum(TR[i])

    print("Tik: (",Tik.shape[0],",",Tik.shape[1],")")
    
    # Document Representation
    DR = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(X[i] != 0)[0] # extrac idxs =! 0
        a = X[i][termos] / DL[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DR[i] = sum(np.multiply(a,Tik[termos])) # alpha * tik; alpha is the df of each term
    # Some of then are empy sentence_vectors[i]  examp 8825 "Nadie se salva de la regla 34 xd"    return Tik
    
    # Document Test Representation
    DLT = np.sum(XT, axis=1)
    DLT[DLT == 0] = 1
    num_docs = XT.shape[0]
    DRT = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(XT[i] != 0)[0] # extrac idxs =! 0
        a = XT[i][termos] / DLT[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DRT[i] = sum(np.multiply(a,Tik[termos]))

    return DR,DRT

In [9]:
DR,DRT = csa (X_train,Y_train,X_test)
print("DR: (",DR.shape[0],",",DR.shape[1],")")

PRB: ( 3810 , 2 )
Tik: ( 3810 , 2 )
DR: ( 6182 , 2 )


In [10]:
#df = pd.DataFrame()
#df  = pd.read_csv('Atest_0.csv',encoding='utf-8',header=None)
#df.head()
#DRT = np.asarray(df)
#print("Matrix of dimension: (",DRT.shape[0],",",DRT.shape[1],")")

#df  = pd.read_csv('nYtest.csv',encoding='utf-8',header=None)
#Y_test = df[0]
#Y_test[Y_test == -1] = 0
#df.head()

In [15]:
for i in DRT:
    print(i)

[0.57950122 0.42049878]
[0.51154591 0.48845409]
[0.49448833 0.50551167]
[0.53203607 0.46796393]
[0.5344956 0.4655044]
[0.5538147 0.4461853]
[0.55763981 0.44236019]
[0.48907685 0.51092315]
[0.53029608 0.46970392]
[0.56048726 0.43951274]
[0.55673431 0.44326569]
[0.53725579 0.46274421]
[0.51935701 0.48064299]
[0.54367927 0.45632073]
[0.53000178 0.46999822]
[0.51550254 0.48449746]
[0.54923552 0.45076448]
[0.56782464 0.43217536]
[0.54776968 0.45223032]
[0.54301447 0.45698553]
[0.5237408 0.4762592]
[0.5638641 0.4361359]
[0.48795085 0.51204915]
[0.44281564 0.55718436]
[0.49074519 0.50925481]
[0.56854518 0.43145482]
[0.50449932 0.49550068]
[0.48480918 0.51519082]
[0.57543782 0.42456218]
[0.45060215 0.54939785]
[0.48811687 0.51188313]
[0.56026486 0.43973514]
[0.50951522 0.49048478]
[0.55504497 0.44495503]
[0.57915905 0.42084095]
[0.44307064 0.55692936]
[0.45386923 0.54613077]
[0.44793582 0.55206418]
[0.55690061 0.44309939]
[0.5075821 0.4924179]
[0.49860224 0.50139776]
[0.55995767 0.44004233]
[0

[0.56953594 0.43046406]
[0.50903064 0.49096936]
[0.4269577 0.5730423]
[0.52394028 0.47605972]
[0.33906714 0.66093286]
[0.52245865 0.47754135]
[0.54218176 0.45781824]
[0.52175489 0.47824511]
[0.56254359 0.43745641]
[0.54866578 0.45133422]
[0.48751441 0.51248559]
[0.58019679 0.41980321]
[0.56208067 0.43791933]
[0.55000746 0.44999254]
[0.56208598 0.43791402]
[0.5210413 0.4789587]
[0.54859121 0.45140879]
[0.51399263 0.48600737]
[0.48872993 0.51127007]
[0.49856461 0.50143539]
[0.46842835 0.53157165]
[0.51929612 0.48070388]
[0.56651427 0.43348573]
[0.52139448 0.47860552]
[0.51546559 0.48453441]
[0.57105179 0.42894821]
[0.46805157 0.53194843]
[0.5445508 0.4554492]
[0.51670015 0.48329985]
[0.57030001 0.42969999]
[0.31378511 0.68621489]
[0.5648978 0.4351022]
[0.52160773 0.47839227]
[0.60156882 0.39843118]
[0.56313557 0.43686443]
[0.51260653 0.48739347]
[0.49515724 0.50484276]
[0.53197168 0.46802832]
[0.55902539 0.44097461]
[0.52108482 0.47891518]
[0.43760909 0.56239091]
[0.52487932 0.47512068]


[0.51738607 0.48261393]
[0.55875232 0.44124768]
[0.42861502 0.57138498]
[0.53027127 0.46972873]
[0.53410689 0.46589311]
[0.57870755 0.42129245]
[0.58490136 0.41509864]
[0.57831711 0.42168289]
[0.54371708 0.45628292]
[0.48027369 0.51972631]
[0.56445763 0.43554237]
[0.43271994 0.56728006]
[0.4910129 0.5089871]
[0.44398873 0.55601127]
[0.50173471 0.49826529]
[0.48938697 0.51061303]
[0.49715653 0.50284347]
[0.53336557 0.46663443]
[0.52615212 0.47384788]
[0.54137476 0.45862524]
[0.52793863 0.47206137]
[0.51871092 0.48128908]
[0.54759258 0.45240742]
[0.48394853 0.51605147]
[0.55695053 0.44304947]
[0.55073469 0.44926531]
[0.56468597 0.43531403]
[0.58552996 0.41447004]
[0.55081734 0.44918266]
[0.61206792 0.38793208]
[0.50332363 0.49667637]
[0.52106522 0.47893478]
[0.5394757 0.4605243]
[0.37744235 0.62255765]
[0.55615149 0.44384851]
[0.5236553 0.4763447]
[0.5564675 0.4435325]
[0.4616249 0.5383751]
[0.58928128 0.41071872]
[0.49808686 0.50191314]
[0.62242543 0.37757457]
[0.51865361 0.48134639]
[0

[0.52401297 0.47598703]
[0.5760176 0.4239824]
[0.5536526 0.4463474]
[0.20916633 0.79083367]
[0.54375856 0.45624144]
[0.4883583 0.5116417]
[0.48583545 0.51416455]
[0.54985719 0.45014281]
[0.5818442 0.4181558]
[0.56462565 0.43537435]
[0.51445166 0.48554834]
[0.66726064 0.33273936]
[0.54783181 0.45216819]
[0.5363681 0.4636319]
[0.54468547 0.45531453]
[0.46235195 0.53764805]
[0.345667 0.654333]
[0.50093065 0.49906935]
[0.54318269 0.45681731]
[0.52649416 0.47350584]
[0.47904577 0.52095423]
[0.57038524 0.42961476]
[0.63549177 0.36450823]
[0.54024092 0.45975908]
[0.54084495 0.45915505]
[0.571737 0.428263]
[0.40806349 0.59193651]
[0.57053358 0.42946642]
[0.58896761 0.41103239]
[0.57256695 0.42743305]
[0.53977272 0.46022728]
[0.53787366 0.46212634]
[0.47812345 0.52187655]
[0.39989858 0.60010142]
[0.51580989 0.48419011]
[0.50089596 0.49910404]
[0.47184221 0.52815779]
[0.51536569 0.48463431]
[0.56475781 0.43524219]
[0.52601145 0.47398855]
[0.58951906 0.41048094]
[0.55123873 0.44876127]
[0.5492098

[0.54699777 0.45300223]
[0.31856942 0.68143058]
[0.54883997 0.45116003]
[0.54409041 0.45590959]
[0.53040215 0.46959785]
[0.52402521 0.47597479]
[0.48279213 0.51720787]
[0.53094229 0.46905771]
[0.63343152 0.36656848]
[0.56543373 0.43456627]
[0.52434448 0.47565552]
[0.50542145 0.49457855]
[0.33655618 0.66344382]
[0.36161517 0.63838483]
[0.5270343 0.4729657]
[0.52452402 0.47547598]
[0.54764542 0.45235458]
[0.55642633 0.44357367]
[0.59800826 0.40199174]
[0.45867186 0.54132814]
[0.53746033 0.46253967]
[0.55057402 0.44942598]
[0.52597481 0.47402519]
[0.4478979 0.5521021]
[0.51242301 0.48757699]
[0.58342468 0.41657532]
[0.52536125 0.47463875]
[0.54593822 0.45406178]
[0.52252005 0.47747995]
[0.49301998 0.50698002]
[0.53032999 0.46967001]
[0.50068958 0.49931042]
[0.51237771 0.48762229]
[0.5048038 0.4951962]
[0.48451707 0.51548293]
[0.53825106 0.46174894]
[0.57974037 0.42025963]
[0.58078358 0.41921642]
[0.44860513 0.55139487]
[0.54336365 0.45663635]
[0.46836158 0.53163842]
[0.24034315 0.75965685

[0.52352713 0.47647287]
[0.48323948 0.51676052]
[0.55110735 0.44889265]
[0.57725552 0.42274448]
[0.50417487 0.49582513]
[0.4923327 0.5076673]
[0.53555805 0.46444195]
[0.47073567 0.52926433]
[0.56013119 0.43986881]
[0.57059947 0.42940053]
[0.5599922 0.4400078]
[0.53065849 0.46934151]
[0.569752 0.430248]
[0.54162983 0.45837017]
[0.42844825 0.57155175]
[0.53989507 0.46010493]
[0.5627615 0.4372385]
[0.57112183 0.42887817]
[0.53270821 0.46729179]
[0.45067674 0.54932326]
[0.54155816 0.45844184]
[0.56464173 0.43535827]
[0.53011812 0.46988188]
[0.5566951 0.4433049]
[0.54433385 0.45566615]
[0.54284307 0.45715693]
[0.55006273 0.44993727]
[0.50182722 0.49817278]
[0.49709768 0.50290232]
[0.55081944 0.44918056]
[0.52022361 0.47977639]
[0.56227407 0.43772593]
[0.54066672 0.45933328]
[0.52968768 0.47031232]
[0.40473377 0.59526623]
[0.49249934 0.50750066]
[0.58268568 0.41731432]
[0.58348341 0.41651659]
[0.47551746 0.52448254]
[0.4943676 0.5056324]
[0.46855895 0.53144105]
[0.49915999 0.50084001]
[0.519

In [16]:
DRT[2159]

array([0.53373212, 0.46626788])

In [11]:
# Create NN model
num_class = 2
model = Sequential()
# Add model layers
#model.add(Dense(10, activation='sigmoid', input_shape=(num_class,)))


model.add(Dense(10, input_dim=num_class,
                kernel_regularizer=regularizers.l2(1e-14),
                activity_regularizer=regularizers.l1(1e-14)))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
#opt = keras.optimizers.SGD(decay=1e-14)
model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model.fit(DR, Y_train, epochs=150, batch_size=10, validation_data=(DRT, Y_test))

In [ ]:
np.setdiff1d(a, b)
sTest averigua quien essss?? y ya n grmas wwy

In [36]:
retains = [0.01,0.06,0.11,0.16,0.21,0.26,0.31,0.36,0.41,0.46,0.51,0.56,0.61,0.66,0.71,0.76,0.81,0.86,0.91,1]
#retains = [0.91]
#retains = [0.110000000000000,0.160000000000000,0.210000000000000,0.260000000000000,0.310000000000000,0.360000000000000,0.410000000000000,0.460000000000000,0.510000000000000,0.560000000000000,0.610000000000000,0.660000000000000,0.710000000000000,0.760000000000000,0.810000000000000,0.860000000000000,0.910000000000000,1]

for j in range (0,len(retains)):
    print("---------------------------------------- ",retains[j]," ---------------------------------------------")
    rXtest = np.zeros((X_test.shape[0], X_test.shape[1]))
    for i in range(0,X_test.shape[0]): # iterating each test document
        noz = np.where(S_test[i] != 0)[0] # from our idx (ONLY INDXS) document matriz, extract idx != 0 (this step could be avoid if we do not agregate those 0)
        #ntermssf = round(len(noz)*retains[j]) # We obtain the number of terns with x% of longitud
        aa = S_test[i][noz]
        aa = np.unique(aa)
        aa = np.sort(aa)
        ntermssf = round(len(aa)*retains[j]) #This was the right thing, analize
        myox = 1 # minnum Termos = 1
        wdix = 0 # 
        freqtsof = np.zeros(X_test.shape[1])  
        # This vector activate the terms present in the Document until that longitud
        while myox <= ntermssf:
            if S_test[i,wdix] != 0:
                freqtsof[int(S_test[i,wdix])-1] += 1 #If the same word appears more than once , just add one to the position
                myox += 1
            wdix += 1
        rXtest[i] = freqtsof
        
    DRS,DRTS = csa(X_train,Y_train,rXtest)
    

    #for i in DRTS:
      #  print(i)
    #print("............")
    #print(DRS[555])
    #print(DRTS[555])
    #DRS = np.nan_to_num(DRS)
    #DR_TS = np.nan_to_num(DR_TS)
    #print(DRS.shape)
    #print(DR_TS.shape)
    model.fit(DRS, Y_train, epochs=10, batch_size=10, validation_data=(DRTS, Y_test))
    #Xtestv = rXtest

----------------------------------------  0.01  ---------------------------------------------
PRB: ( 3810 , 2 )
Tik: ( 3810 , 2 )
Train on 6182 samples, validate on 2647 samples
Epoch 1/10
6182/6182 [==============================] - 1s 173us/step - loss: 0.3970 - acc: 0.8208 - val_loss: 1.1823 - val_acc: 0.4329
Epoch 2/10
6182/6182 [==============================] - 1s 156us/step - loss: 0.3964 - acc: 0.8192 - val_loss: 1.1257 - val_acc: 0.4367
Epoch 3/10
6182/6182 [==============================] - 1s 154us/step - loss: 0.3964 - acc: 0.8179 - val_loss: 1.1396 - val_acc: 0.4337
Epoch 4/10
6182/6182 [==============================] - 1s 154us/step - loss: 0.3967 - acc: 0.8190 - val_loss: 1.1344 - val_acc: 0.4367
Epoch 5/10
6182/6182 [==============================] - 1s 156us/step - loss: 0.3966 - acc: 0.8151 - val_loss: 1.1521 - val_acc: 0.4333
Epoch 6/10
6182/6182 [==============================] - 1s 154us/step - loss: 0.3968 - acc: 0.8182 - val_loss: 1.1515 - val_acc: 0.4333
Epoch 

6182/6182 [==============================] - 1s 155us/step - loss: 0.3966 - acc: 0.8162 - val_loss: 0.6775 - val_acc: 0.7427
Epoch 4/10
6182/6182 [==============================] - 1s 154us/step - loss: 0.3968 - acc: 0.8172 - val_loss: 0.6705 - val_acc: 0.7431
Epoch 5/10
6182/6182 [==============================] - 1s 153us/step - loss: 0.3971 - acc: 0.8188 - val_loss: 0.6780 - val_acc: 0.7431
Epoch 6/10
6182/6182 [==============================] - 1s 154us/step - loss: 0.3966 - acc: 0.8170 - val_loss: 0.6762 - val_acc: 0.7416
Epoch 7/10
6182/6182 [==============================] - 1s 154us/step - loss: 0.3968 - acc: 0.8183 - val_loss: 0.6746 - val_acc: 0.7412
Epoch 8/10
6182/6182 [==============================] - 1s 157us/step - loss: 0.3966 - acc: 0.8154 - val_loss: 0.6995 - val_acc: 0.7314
Epoch 9/10
6182/6182 [==============================] - 1s 157us/step - loss: 0.3966 - acc: 0.8196 - val_loss: 0.6731 - val_acc: 0.7416
Epoch 10/10
6182/6182 [==============================] - 1s

Epoch 7/10
6182/6182 [==============================] - 1s 155us/step - loss: 0.3965 - acc: 0.8179 - val_loss: 0.5561 - val_acc: 0.7612
Epoch 8/10
6182/6182 [==============================] - 1s 157us/step - loss: 0.3963 - acc: 0.8166 - val_loss: 0.5456 - val_acc: 0.7662
Epoch 9/10
6182/6182 [==============================] - 1s 156us/step - loss: 0.3966 - acc: 0.8170 - val_loss: 0.5473 - val_acc: 0.7646
Epoch 10/10
6182/6182 [==============================] - 1s 163us/step - loss: 0.3963 - acc: 0.8164 - val_loss: 0.5501 - val_acc: 0.7639
----------------------------------------  0.56  ---------------------------------------------
PRB: ( 3810 , 2 )
Tik: ( 3810 , 2 )
Train on 6182 samples, validate on 2647 samples
Epoch 1/10
6182/6182 [==============================] - 1s 177us/step - loss: 0.3967 - acc: 0.8175 - val_loss: 0.5318 - val_acc: 0.7745
Epoch 2/10
6182/6182 [==============================] - 1s 164us/step - loss: 0.3963 - acc: 0.8162 - val_loss: 0.5303 - val_acc: 0.7760
Epoch

6182/6182 [==============================] - 1s 157us/step - loss: 0.3965 - acc: 0.8195 - val_loss: 0.4716 - val_acc: 0.7971
----------------------------------------  0.81  ---------------------------------------------
PRB: ( 3810 , 2 )
Tik: ( 3810 , 2 )
Train on 6182 samples, validate on 2647 samples
Epoch 1/10
6182/6182 [==============================] - 1s 172us/step - loss: 0.3971 - acc: 0.8156 - val_loss: 0.4572 - val_acc: 0.8032
Epoch 2/10
6182/6182 [==============================] - 1s 155us/step - loss: 0.3965 - acc: 0.8169 - val_loss: 0.4571 - val_acc: 0.8039
Epoch 3/10
6182/6182 [==============================] - 1s 157us/step - loss: 0.3967 - acc: 0.8164 - val_loss: 0.4602 - val_acc: 0.8017
Epoch 4/10
6182/6182 [==============================] - 1s 164us/step - loss: 0.3961 - acc: 0.8172 - val_loss: 0.4717 - val_acc: 0.7956
Epoch 5/10
6182/6182 [==============================] - 1s 155us/step - loss: 0.3970 - acc: 0.8164 - val_loss: 0.4588 - val_acc: 0.7998
Epoch 6/10
6182/6

In [ ]:
Comparar los DR DRT :)
    eso esta ok, matplab llena con 1 valores incorrectos ...jummm
ahora vamos a comparar los mini dr y drt  , el error tiene que estar en algun lado muajajaj
probar matlab con 10 iter :)
    
    comparar los acc con 10 ii
    y ver hasta donde esta tomando por uq no toma en cuenta todo S ....